# Загрузка данных + импорты

## Импорты

In [53]:
#импорты
#from nltk.tokenize import word_tokenize, RegexpTokenizer
#from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.naive_bayes import ComplementNB
#from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold


In [18]:
FILESOURCE = 'SMSSpamCollection.txt' # файл источник


## Чтение файла

In [19]:
# читаем файл
with open(FILESOURCE) as f: 
    full_file = f.read()
print(full_file[:20])

ham	Go until jurong 


**Вывод:** данные удалось прочитать, уже неплохо

# Предобработка

In [20]:
# нижний регистр и разделение на строки.
full_file = full_file.lower()
lst = full_file.splitlines()

In [21]:
# Отделение таргета
lst_with_target = []
for line in lst:
    lst_tmp = []
    lst_line = line.split('\t')  
    if lst_line[0] == 'spam' or lst_line[0] == 'ham':
        lst_tmp.append('ok')
        lst_tmp.append(lst_line[0])
        lst_tmp.append(lst_line[1:])
        lst_with_target.append(lst_tmp)
    else:
        lst_tmp.append('not_ok')
        lst_tmp.append(lst_line[0])
        lst_tmp.append(lst_line[1:])
        lst_with_target.append(lst_tmp)

In [22]:
# проверка на структуру файла
for i in range(len(lst_with_target)):
    if lst_with_target[i][0] != 'ok':
        print(lst_with_target[i], 'is not ok')
    if len(lst_with_target[i]) > 3:
        print(lst_with_target[i], '>3')

**Вывод:** структура файла соблюдается

In [23]:
vectorizer = CountVectorizer(stop_words='english', token_pattern='[a-z]\w+')
X = vectorizer.fit_transform(lst)
#vectorizer.get_feature_names_out()


In [24]:
vectorizer2 = CountVectorizer(token_pattern='[0-9]\w+')
X2 = vectorizer2.fit_transform(lst)
vectorizer3 = CountVectorizer(token_pattern='[§!@#$%^&*()_+-<>:"|\?~{}]')
X3 = vectorizer3.fit_transform(lst)


In [25]:
df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names_out())
df2 = pd.DataFrame(X2.toarray(), columns= vectorizer2.get_feature_names_out())
df3 = pd.DataFrame(X3.toarray(),columns=vectorizer3.get_feature_names_out())

In [26]:
df['words_with_digits'] = df2.T.sum()
df['special_symbols'] = df3.T.sum()

In [27]:
df['target'] =  (df['spam'] > 0).apply(int)

df = df.drop(['spam', 'ham'], axis=1)

In [28]:
len(vectorizer.get_feature_names_out())

7680

In [29]:
# Создаем транспонированый df
df_t = df.T

## Среднее количество слов в сообщении

In [30]:
df[df['target'] == 1].T.sum().mean()

39.163319946452475

In [31]:
df[df['target'] == 0].T.sum().mean()

10.946550652579242

In [32]:
# разделение
features = df.drop('target',axis=1)
target = df['target']
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size=.3, random_state=123456)
print(features_train.shape, features_test.shape, target_train.shape, target_test.shape )

(3901, 7680) (1673, 7680) (3901,) (1673,)


In [33]:
df['len_diff'] = abs(df_t.sum() - features_train[target_train == 1].T.sum().mean())

### Сбалансированный df

In [34]:
features_train_ones = features_train[target_train == 1]

In [35]:
features_train_zeros = features_train[target_train == 0].sample(n=features_train_ones.shape[0])

In [36]:
features_train_balanced = pd.concat([features_train_ones, features_train_zeros])

In [37]:
features_train_balanced = features_train_balanced.join(target_train)
features_train_balanced = shuffle(features_train_balanced)

In [38]:
target_train_balanced = features_train_balanced['target']

In [39]:
features_train_balanced = features_train_balanced.drop('target', axis=1)

# EDA

## Баланс классов

### Общий баланс классов

In [40]:
df['target'].mean()

0.1340150699677072

### Баланс классов train

In [41]:
target_train.mean()

0.12971033068443988

In [42]:
target_train_balanced.mean()

0.5

### Баланс классов test

In [43]:
target_test.mean()

0.14405260011954574

## Поиск важных слов


In [44]:
df[df['target'] == 1].sum().sort_values(ascending=False)[:25]

special_symbols      15925.000000
len_diff              8036.452569
words_with_digits     2540.000000
target                 747.000000
free                   226.000000
txt                    166.000000
ur                     144.000000
mobile                 128.000000
text                   125.000000
stop                   125.000000
claim                  113.000000
reply                  104.000000
www                     98.000000
prize                   93.000000
just                    79.000000
cash                    76.000000
won                     76.000000
uk                      74.000000
send                    71.000000
new                     69.000000
nokia                   67.000000
win                     64.000000
urgent                  63.000000
tone                    60.000000
week                    60.000000
dtype: float64

In [45]:
df[df['target'] == 0].sum().sort_values(ascending=False)[:25]

len_diff             132892.859684
special_symbols       18803.000000
words_with_digits       356.000000
gt                      318.000000
lt                      316.000000
just                    297.000000
ok                      288.000000
ll                      266.000000
ur                      247.000000
know                    237.000000
good                    235.000000
like                    234.000000
got                     234.000000
come                    230.000000
day                     217.000000
love                    205.000000
time                    202.000000
going                   169.000000
want                    165.000000
home                    165.000000
lor                     162.000000
don                     159.000000
need                    159.000000
sorry                   157.000000
da                      150.000000
dtype: float64

**Вывод:** Можно утверждать, что в spam-сообщениях часто употребляются такие слова как free и txt, а также есть много слов с цифрами

## Модели

In [46]:
cnb = ComplementNB()
cnb_bal = ComplementNB()
lgr = LogisticRegression(penalty='l2')
cbc = CatBoostClassifier()

In [47]:
cnb.fit(features_train,target_train)
cnb_bal.fit(features_train_balanced, target_train_balanced)
lgr.fit(features_train_balanced, target_train_balanced)
cbc.fit(features_train_balanced, target_train_balanced, verbose=0)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [48]:
def print_model_rep(model_kind, model, features_train, target_train, features_test, target_test):
    print(model_kind)
    print('train')
    prediction = model.predict(features_train)
    print(classification_report(target_train, prediction))
    print('test')
    prediction = model.predict(features_test)
    print(classification_report(target_test, prediction))



In [49]:
print_model_rep('NB not balanced', cnb, features_train, target_train, features_test, target_test)

NB not balanced
train
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      3395
           1       0.80      0.99      0.89       506

    accuracy                           0.97      3901
   macro avg       0.90      0.98      0.93      3901
weighted avg       0.97      0.97      0.97      3901

test
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1432
           1       0.68      0.98      0.80       241

    accuracy                           0.93      1673
   macro avg       0.84      0.95      0.88      1673
weighted avg       0.95      0.93      0.94      1673



In [50]:
print_model_rep('NB balanced', cnb_bal, features_train_balanced, target_train_balanced, features_test, target_test)

NB balanced
train
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       506
           1       0.91      0.99      0.95       506

    accuracy                           0.95      1012
   macro avg       0.95      0.95      0.95      1012
weighted avg       0.95      0.95      0.95      1012

test
              precision    recall  f1-score   support

           0       1.00      0.85      0.92      1432
           1       0.52      0.98      0.68       241

    accuracy                           0.87      1673
   macro avg       0.76      0.92      0.80      1673
weighted avg       0.93      0.87      0.88      1673



In [51]:
print_model_rep('log_reg_balanced', lgr, features_train_balanced, target_train_balanced, features_test, target_test)


log_reg_balanced
train
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       506
           1       1.00      0.99      0.99       506

    accuracy                           0.99      1012
   macro avg       0.99      0.99      0.99      1012
weighted avg       0.99      0.99      0.99      1012

test
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1432
           1       0.94      0.93      0.94       241

    accuracy                           0.98      1673
   macro avg       0.96      0.96      0.96      1673
weighted avg       0.98      0.98      0.98      1673



In [52]:
print_model_rep('catboost', cbc, features_train_balanced, target_train_balanced, features_test, target_test)


catboost
train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       506
           1       1.00      1.00      1.00       506

    accuracy                           1.00      1012
   macro avg       1.00      1.00      1.00      1012
weighted avg       1.00      1.00      1.00      1012

test
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1432
           1       0.84      0.94      0.89       241

    accuracy                           0.97      1673
   macro avg       0.92      0.95      0.93      1673
weighted avg       0.97      0.97      0.97      1673



## Кросс-валидация

In [55]:
kf = KFold(n_splits=5)
kf.get_n_splits()

In [73]:
def get_cv_values(model_kind, model, features, target, kf ):
    num_fold = 0
    for train_index, test_index in kf.split(features):
        num_fold += 1
        print(f'Fold number {num_fold}')
        print_model_rep(model_kind, model, features.loc[train_index], target[train_index], features.loc[test_index], target[test_index])

In [75]:
get_cv_values('NB not balanced', cnb, features, target, kf)

Fold number 1
NB not balanced
train
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      3880
           1       0.75      0.99      0.85       579

    accuracy                           0.96      4459
   macro avg       0.87      0.97      0.91      4459
weighted avg       0.97      0.96      0.96      4459

test
              precision    recall  f1-score   support

           0       1.00      0.95      0.98       947
           1       0.79      0.99      0.88       168

    accuracy                           0.96      1115
   macro avg       0.89      0.97      0.93      1115
weighted avg       0.97      0.96      0.96      1115

Fold number 2
NB not balanced
train
              precision    recall  f1-score   support

           0       1.00      0.95      0.97      3853
           1       0.75      0.98      0.85       606

    accuracy                           0.95      4459
   macro avg       0.88      0.97      0.91      44

In [76]:
get_cv_values('NB balanced', cnb_bal, features, target, kf)

Fold number 1
NB balanced
train
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      3880
           1       0.48      0.99      0.65       579

    accuracy                           0.86      4459
   macro avg       0.74      0.92      0.78      4459
weighted avg       0.93      0.86      0.88      4459

test
              precision    recall  f1-score   support

           0       1.00      0.84      0.91       947
           1       0.52      0.98      0.68       168

    accuracy                           0.86      1115
   macro avg       0.76      0.91      0.80      1115
weighted avg       0.93      0.86      0.88      1115

Fold number 2
NB balanced
train
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      3853
           1       0.49      0.99      0.66       606

    accuracy                           0.86      4459
   macro avg       0.75      0.91      0.79      4459
weigh

In [77]:
get_cv_values('log_reg_balanced', lgr, features, target, kf)



Fold number 1
log_reg_balanced
train
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3880
           1       0.91      0.97      0.94       579

    accuracy                           0.98      4459
   macro avg       0.95      0.98      0.97      4459
weighted avg       0.98      0.98      0.98      4459

test
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       947
           1       0.94      0.97      0.96       168

    accuracy                           0.99      1115
   macro avg       0.97      0.98      0.97      1115
weighted avg       0.99      0.99      0.99      1115

Fold number 2
log_reg_balanced
train
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3853
           1       0.92      0.97      0.95       606

    accuracy                           0.98      4459
   macro avg       0.96      0.98      0.97      

In [78]:
get_cv_values('catboost', cbc, features, target, kf)

Fold number 1
catboost
train
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      3880
           1       0.84      0.98      0.91       579

    accuracy                           0.97      4459
   macro avg       0.92      0.98      0.95      4459
weighted avg       0.98      0.97      0.97      4459

test
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       947
           1       0.92      0.98      0.95       168

    accuracy                           0.98      1115
   macro avg       0.96      0.98      0.97      1115
weighted avg       0.98      0.98      0.98      1115

Fold number 2
catboost
train
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      3853
           1       0.87      0.98      0.92       606

    accuracy                           0.98      4459
   macro avg       0.93      0.98      0.95      4459
weighted av

**Вывод** нам удалось сделать модель, которая в некоторой степени умеет классифицировать сообщения на SPAM и HAM